# Challenge 6.0

This notebook aims to respond to point `6.0` of the script `SISTCA_2023-24_Lab_Script_Template_V3_19May2024`.

Based on the previous exercises, the challenge is to choose a region of the globe and process the satellite data. It is proposed that a map of the evolution of a forest fire in a region of Brazil be obtained, for example obtaining the difference between a dataset at an instant before the fire happened and a dataset after it happened. The satellite data is available at [Brasil Data Cube](https://data.inpe.br/bdc/web/).

Installing Packages and Dependencies

In [1]:
import pystac_client

In [2]:
from wtss import *

In [3]:
import numpy 

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt

In [5]:
import rasterio
from rasterio.crs import CRS
from rasterio.warp import transform
from rasterio.windows import from_bounds

Connection to the Brazil Data Cube API

To perform the necessary operations, you must use the `pystac_client` API. To find out all about its features, you can consult [STAC client for Python](https://github.com/brazil-data-cube/stac.py).

In [6]:
# function imported from Brazil Data Cube's GitHub repository

def read(uri: str, bbox: list, masked: bool = True, crs: str = None):
    """Read raster window as numpy.ma.masked_array."""
    source_crs = CRS.from_string('EPSG:4326')
    if crs:
        source_crs = CRS.from_string(crs)

    # Expects the bounding box has 4 values
    w, s, e, n = bbox
        
    with rasterio.open(uri) as dataset:
        transformer = transform(source_crs, dataset.crs, [w, e], [s, n])
        window = from_bounds(transformer[0][0], transformer[1][0], 
                             transformer[0][1], transformer[1][1], dataset.transform)
        return dataset.read(1, window=window, masked=masked)
    

In [7]:
# function imported from Brazil Data Cube's GitHub repository

def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

In [8]:
token = 'r5p7J7kpqIbnPY47jX4i8YfaVcKq7iPFsO3nQQ6OpF' # Change token

In [9]:
parameters = dict(access_token=token) # Change token
service_STAC = pystac_client.Client.open('https://brazildatacube.dpi.inpe.br/stac/', parameters=parameters)

In [10]:
service_WTSS = WTSS('https://brazildatacube.dpi.inpe.br/', 
               access_token=token) # Change token

# We only connect to the WTSS service because the API for listing available bands

List All Available Bands and Timelines on Sentinel 2

In [11]:
satelite = 'S2-16D-2'  

In [12]:
service_WTSS[satelite] 

name,common name,description,datatype,valid range,scale,nodata
B12,swir22,,int16,"{'min': 0.0, 'max': 10000.0}",0.0001,0.0
SCL,quality,,uint8,"{'min': 0.0, 'max': 11.0}",1.0,0.0
CLEAROB,ClearOb,Clear Observation Count,uint8,"{'min': 1.0, 'max': 255.0}",1.0,0.0
NDVI,ndvi,,int16,"{'min': -10000.0, 'max': 10000.0}",0.0001,-9999.0
NBR,nbr,,int16,"{'min': -10000.0, 'max': 10000.0}",0.0001,-9999.0
EVI,evi,Enhanced Vegetation Index,int16,"{'min': -10000.0, 'max': 10000.0}",0.0001,-9999.0
B01,coastal,,int16,"{'min': 0.0, 'max': 10000.0}",0.0001,0.0
B02,blue,,int16,"{'min': 0.0, 'max': 10000.0}",0.0001,0.0
B03,green,,int16,"{'min': 0.0, 'max': 10000.0}",0.0001,0.0
B04,red,,int16,"{'min': 0.0, 'max': 10000.0}",0.0001,0.0


Then you can search for the features you want

In [13]:
bbox = (-50.5600,-11.0600,-50.4500,-10.9600)   #localização a utilizar

In [14]:
service_STAC.get_collection(satelite).get_items()  

item_search = service_STAC.search(bbox=bbox,
                             datetime='2021-04-01/2021-07-30',
                             collections=[satelite])

item_search.matched() #numero de ficheiros de informação disponiveis com estas defenições

#bbox = dimensoes do quadrado de pesquisa a obter em conjunto com o mapa do BDC
#datetime = linha temporal pertendida a obter os ficheiros de informação
#collections= satelite que ira fornecer a informação

9

In [15]:
counter = 0
for item in item_search.get_items():    #listagem de ficheiros de informação disponiveis 
    print(f"[{counter}].  {item}")
    counter += 1

[0].  <Item id=S2-16D_V2_026017_20210728>
[1].  <Item id=S2-16D_V2_026017_20210712>
[2].  <Item id=S2-16D_V2_026017_20210626>
[3].  <Item id=S2-16D_V2_026017_20210610>
[4].  <Item id=S2-16D_V2_026017_20210525>
[5].  <Item id=S2-16D_V2_026017_20210509>
[6].  <Item id=S2-16D_V2_026017_20210423>
[7].  <Item id=S2-16D_V2_026017_20210407>
[8].  <Item id=S2-16D_V2_026017_20210322>


In [16]:
items = list(item_search.get_items()) # We only connect to the WTSS service because the API for listing available bands

In [17]:
first_item = items[8]               #Choosing the images to use 
midle_item = items[4]
second_item = items[0]

Read the necessary bands for NBR and for RGB plots

In [18]:
red_first = read(first_item.assets['B04'].href, bbox=bbox)
green_first = read(first_item.assets['B03'].href, bbox=bbox)
blue_first = read(first_item.assets['B02'].href, bbox=bbox)

In [ ]:
red_second = read(second_item.assets['B04'].href, bbox=bbox)
green_second = read(second_item.assets['B03'].href, bbox=bbox)
blue_second = read(second_item.assets['B02'].href, bbox=bbox)

In [ ]:
NIR_first_image = read(first_item.assets['B08'].href, bbox=bbox)      
NIR_midle_image = read(midle_item.assets['B08'].href, bbox=bbox)
NIR_second_image = read(second_item.assets['B08'].href, bbox=bbox)

In [ ]:
SWIR_first_image = read(first_item.assets['B12'].href, bbox=bbox)    
SWIR_midle_image = read(midle_item.assets['B12'].href, bbox=bbox)
SWIR_second_image = read(second_item.assets['B12'].href, bbox=bbox)

## Calculate the NBR for all desired datasets 




$$
NBR = \frac{(NIR - SWIR)}{(NIR + SWIR)}
$$
$$
$$
<center><b>Equation 1</b> - SWIR.</center>

In [ ]:
NBR_first=(NIR_first_image-SWIR_first_image)/(NIR_first_image+SWIR_first_image)
NBR_midle=(NIR_midle_image-SWIR_midle_image)/(NIR_midle_image+SWIR_midle_image)
NBR_second=(NIR_second_image-SWIR_second_image)/(NIR_second_image+SWIR_second_image)


NBR_diff =  NBR_second - NBR_first

Now we can plot all the diferent tiles 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12, 10))

ax1.imshow(NBR_first, cmap='RdBu')
ax1.set_title('NBR before the fire')

ax2.imshow(NBR_midle, cmap='RdBu')
ax2.set_title('NBR midle of the Fire')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12, 10))

ax1.imshow(NBR_second, cmap='RdBu')
ax1.set_title('NBR Post Fire')

im = ax2.imshow(NBR_diff, cmap='RdBu', vmin=-1, vmax=1)
ax2.set_title('Consumed Area')

In the images we can see the effects of the fire in the region and we can assume that the red area `NBR Post Fire` is the burned area.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 1))
fig.subplots_adjust(bottom=0.5, top=1)  


cbar = fig.colorbar(im, ax, orientation='horizontal')
cbar.set_label('NBR Value')

ax.text(0, -0.7, 'Vegetação Queimada', ha='center', va='top', transform=ax.transAxes, fontsize=12)
ax.text(1, -0.7, 'Vegetação Saudável', ha='center', va='top', transform=ax.transAxes, fontsize=12)

Then we can plot the rgb images 

In [ ]:
red_first = normalize(red_first)
green_first = normalize(green_first)
blue_first = normalize(blue_first)

rgb_first  = numpy.dstack((red_first, green_first, blue_first))


 
fig, ax = plt.subplots(figsize=(12, 10))
ax.set_title('Imagem RGB pré fogo Normalizada')
ax.imshow(rgb_first)



In [ ]:
red_second = normalize(red_second)
green_second = normalize(green_second)
blue_second = normalize(blue_second)

rgb_second = numpy.dstack((red_second, green_second, blue_second))

fig, ax2 = plt.subplots(figsize=(12, 10))
ax2.set_title('Imagem RGB pos fogo Normalizada')
ax2.imshow(rgb_second)

